In [1]:
import numpy as np
import pandas as pd
import joblib

# ============================
# 1. Load trained models
# ============================

scaler = joblib.load("scaler.joblib")
kmeans = joblib.load("temporal_kmeans.joblib")

print("Models loaded successfully")

# ============================
# 2. Load new traffic data
# ============================
# Expected format:
# rows = timestamps
# columns = sensor IDs (same as METR-LA)

df_new = pd.read_hdf("METR-LA.h5", key="df")

# Preprocess
df_new = df_new.interpolate().bfill()

# ============================
# 3. Feature extraction (same as training)
# ============================

def extract_temporal_features(df):
    features = []

    for sensor in df.columns:
        s = df[sensor]

        features.append([
            sensor,
            s.mean(),
            s.std(),
            s.diff().mean(),
            s.quantile(0.95) - s.quantile(0.05)
        ])

    return pd.DataFrame(
        features,
        columns=[
            "sensor_id",
            "mean_speed",
            "speed_std",
            "avg_change",
            "speed_range"
        ]
    )

features_df = extract_temporal_features(df_new)

# ============================
# 4. Scale + Predict clusters
# ============================

X = features_df.drop(columns=["sensor_id"])
X_scaled = scaler.transform(X)

cluster_labels = kmeans.predict(X_scaled)
features_df["temporal_cluster"] = cluster_labels

# ============================
# 5. Map clusters → traffic state
# ============================

traffic_names = {
    0: "Free-flow",
    1: "Moderate",
    2: "Peak-heavy",
    3: "Unstable"
}

signal_policy = {
    "Free-flow": "Short cycle, equal green splits",
    "Moderate": "Moderate cycle, biased main-road green",
    "Peak-heavy": "Long cycle, dominant-direction priority",
    "Unstable": "Adaptive / responsive control"
}

features_df["traffic_state"] = features_df["temporal_cluster"].map(traffic_names)
features_df["signal_strategy"] = features_df["traffic_state"].map(signal_policy)

# ============================
# 6. Output
# ============================

print("\nSample inference output:")
print(features_df.head())

features_df.to_csv("inference_output.csv", index=False)

print("\nInference complete. Results saved.")

Models loaded successfully

Sample inference output:
  sensor_id  mean_speed  speed_std  avg_change  speed_range  temporal_cluster  \
0    773869   54.631359  22.619199    0.000021    68.444444                 0   
1    767541   60.452789  15.970239   -0.000021    67.714286                 2   
2    767542   60.726120  18.313353   -0.000010    69.375000                 2   
3    717447   49.524287  15.843261   -0.000008    62.888889                 2   
4    717446   46.079798  19.350345   -0.001951    67.666667                 1   

  traffic_state                          signal_strategy  
0     Free-flow          Short cycle, equal green splits  
1    Peak-heavy  Long cycle, dominant-direction priority  
2    Peak-heavy  Long cycle, dominant-direction priority  
3    Peak-heavy  Long cycle, dominant-direction priority  
4      Moderate   Moderate cycle, biased main-road green  

Inference complete. Results saved.
